In [0]:
import pyspark.sql.functions as sf
from time import time
spark.conf.set("spark.sql.session.timeZone", "America/New_York")

In [0]:
# Read data from 'runescape.02_silver.item_mapping'
df_map = spark.read.table("runescape.02_silver.item_mapping")

In [0]:
# Read data from 'runescape.02_silver.latest_prices_cleansed'
df = spark.read.table("runescape.02_silver.latest_prices_enriched")
df = df.withColumn("NYCTime", sf.from_unixtime('time'))


In [0]:
# Read data from 'runescape.01_bronze.1h_prices' to get Volumes
# TODO replace this read with the new latest_1_prices table that will have the latest 1 hour price for each item
df_avg = spark.read.table("runescape.01_bronze.1h_prices")

In [0]:
df_avg.display()

In [0]:
# Get average of 1 hour price data over the last 24 hours
# TODO replace this with actual 24 hour calculation based on underlying raw data
unix_time_24_hours_ago = time() - 86400
df_24_hours_avg = df_avg.\
    filter(f"time >= {unix_time_24_hours_ago}").\
    groupBy("id").\
    agg(sf.round(sf.avg("avg1HourHigh"), 2).alias("avg_24hr_High")
    , sf.round(sf.avg("avg1HourHighVolume"), 2).alias("avg_24hr_High_Volume")
    , sf.round(sf.avg("avg1HourLow"), 2).alias("avg_24hr_Low")
    , sf.round(sf.avg("avg1HourLowVolume"), 2).alias("avg_24hr_Low_Volume"))




In [0]:
%skip
df_24_hours_avg.display()

In [0]:

# Filter to only use the most recent Volume data
max_time = df_avg.select(sf.max(df_avg.time))

#df_avg = df_avg.filter("time == max_time.first('time'")
df_avg = df_avg.join(max_time, df_avg.time == sf.col("max(time)"))
df_avg = df_avg.drop("max(time)").drop("time")
#df_avg.display()




TODO remove outliers (beyond 3 stdev?)

In [0]:
# filter out low limit items
df9 = df.filter(sf.col("limit") >= 70)

# Data from the last hour only
#df9 = df9.filter((sf.col("time") > sf.unix_timestamp() - 3600))

In [0]:
df10 = df9.groupBy("id","highorlow").agg(sf.round(sf.avg("price"), 2).alias("avg_price"), sf.round(sf.std("price"), 2).alias("std_price"))
#join with volume data
df10 = df10.join(df_map,"id")
df10 = df10.join(df_24_hours_avg, "id")
df10 = df10.join(df_avg, "id").withColumn("std/avg", sf.round(sf.col("std_price")/sf.col("avg_price"), 3))
df10 = df10.withColumn("high_Alch_Margin", sf.round((sf.col("highalch") - sf.col("avg_price")) - 100,1))
df10 = df10.withColumn("1hour_high_price-price", sf.round(sf.col("avg1HourHigh") - sf.col("avg_price"), 2))
df10 = df10.withColumn("price_shift/1hour_high_price", sf.round(sf.col("1hour_high_price-price")/sf.col("avg_price") , 2))
df10 = df10.withColumn("limit_X_std_price", sf.col("std_price")*sf.col("limit"))
df10 = df10.withColumn("1hour_high-24hour_high", sf.round(sf.col("avg1HourHigh") - sf.col("avg_24hr_High"), 2))





df10 = df10.filter("highorlow = 'low'")
df10 = df10.filter("std_price >= 2")
df10 = df10.filter("avg_24hr_High_Volume > 5000")
#df10 = df10.sort("std/avg", ascending=False)
df10 = df10.sort("1hour_high-24hour_high")
df10.display()

In [0]:
%skip
df8.display()